In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, confusion_matrix, classification_report, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from scipy.stats import pearsonr
from sklearn.preprocessing import LabelEncoder
import time

In [2]:
def esa_score(phi, alpha):
    return np.dot(alpha, phi)

def threshold_crossing_rate(esa_baseline, esa_moral, tau):
    crossed = (esa_baseline < tau) & (esa_moral >= tau)
    return np.mean(crossed)

def moral_win_rate(esa_baseline, esa_moral):
    return np.mean(esa_moral > esa_baseline)

def esa_difference(esa_baseline, esa_moral):
    return np.mean(esa_moral - esa_baseline)

In [3]:
def evaluate_classification(y_true, y_pred):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1_score': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred)
    }
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    return metrics

In [4]:
df = pd.read_csv('grad admission - ethics.csv')
    
y = df['accept_status_moral']
X = df.drop(columns=['accept_status', 'ESA', 'CST', 'severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int', 'accept_status_moral'])

esa_features = df[['severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int']].values
tau_values = df['CST'].values
alpha = np.array([0.4, 0.2, 0.3, 0.0, 0.0, 0.1])

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

dnn_baseline_preds = np.zeros(len(X))
dnn_override_preds = np.zeros(len(X))
dnn_penalized_preds = np.zeros(len(X))

def build_dnn_model(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_dim=input_dim),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model



In [5]:
b_time = 0
o_time = 0
p_time = 0

for fold, (train_idx, val_idx) in enumerate(skf.split(X_scaled, y)):
    X_train, X_val = X_scaled.iloc[train_idx], X_scaled.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    b_time_s = time.perf_counter()
    # -------- Baseline --------
    model_baseline = build_dnn_model(X_train.shape[1])
    model_baseline.fit(X_train, y_train, epochs=30, batch_size=32, verbose=0,
                           validation_data=(X_val, y_val),
                           callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])
    y_val_pred_baseline = (model_baseline.predict(X_val).flatten() >= 0.5).astype(int)
    dnn_baseline_preds[val_idx] = y_val_pred_baseline
    print(f"Fold {fold+1} - DNN Baseline:")
    evaluate_classification(y_val, y_val_pred_baseline)
    b_time_e = time.perf_counter()
    b_time = b_time + (b_time_e - b_time_s)

    o_time_s = time.perf_counter()
    # -------- Override --------
    phi_val = esa_features[val_idx]
    tau_val = tau_values[val_idx]
    esa_vals = np.array([esa_score(phi, alpha) for phi in phi_val])
    moral_preds = (esa_vals >= tau_val).astype(int)
    dnn_override_preds[val_idx] = moral_preds
    print(f"Fold {fold+1} - DNN ESA Override:")
    evaluate_classification(y_val, moral_preds)
    o_time_e = time.perf_counter()
    o_time = o_time + (o_time_e - o_time_s)

    p_time_s = time.perf_counter()
    # -------- Penalized --------
    phi_train = esa_features[train_idx]
    tau_train = tau_values[train_idx]
    moral_penalty = np.array([(tau - esa_score(phi, alpha))**2 for phi, tau in zip(phi_train, tau_train)])
    sample_weights = np.clip(1 + 5 * moral_penalty, 1, 10)

    model_penalized = build_dnn_model(X_train.shape[1])
    model_penalized.fit(X_train, y_train, sample_weight=sample_weights, epochs=30, batch_size=32, verbose=0,
                        validation_data=(X_val, y_val),
                        callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])
    y_val_pred_penalized = (model_penalized.predict(X_val).flatten() >= 0.5).astype(int)
    dnn_penalized_preds[val_idx] = y_val_pred_penalized
    print(f"Fold {fold+1} - DNN ESA Penalized:")
    evaluate_classification(y_val, y_val_pred_penalized)
    p_time_e = time.perf_counter()
    p_time = p_time + (p_time_e - p_time_s)

print("\n--- Final Evaluation (DNN Baseline) ---")
#evaluate_classification(y, dnn_baseline_preds)
metrics = evaluate_classification(y, dnn_baseline_preds)
print(metrics)

print("\n--- Final Evaluation (DNN ESA Override) ---")
#evaluate_classification(y, dnn_override_preds)
metrics = evaluate_classification(y, dnn_override_preds)
print(metrics)

print("\n--- Final Evaluation (DNN ESA Penalized) ---")
#evaluate_classification(y, dnn_penalized_preds)
metrics = evaluate_classification(y, dnn_penalized_preds)
print(metrics)

print("time baseline:", b_time)
print("time override:", o_time)
print("time penalized:", p_time)

5/5 [==============================] - 0s 3ms/step
Fold 1 - DNN Baseline:

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        65
           1       1.00      1.00      1.00        89

    accuracy                           1.00       154
   macro avg       1.00      1.00      1.00       154
weighted avg       1.00      1.00      1.00       154


Confusion Matrix:
[[65  0]
 [ 0 89]]
Fold 1 - DNN ESA Override:

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        65
           1       0.58      1.00      0.73        89

    accuracy                           0.58       154
   macro avg       0.29      0.50      0.37       154
weighted avg       0.33      0.58      0.42       154


Confusion Matrix:
[[ 0 65]
 [ 0 89]]


C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

5/5 [==============================] - 0s 3ms/step
Fold 1 - DNN ESA Penalized:

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        65
           1       1.00      1.00      1.00        89

    accuracy                           1.00       154
   macro avg       1.00      1.00      1.00       154
weighted avg       1.00      1.00      1.00       154


Confusion Matrix:
[[65  0]
 [ 0 89]]
5/5 [==============================] - 0s 3ms/step
Fold 2 - DNN Baseline:

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        65
           1       1.00      1.00      1.00        89

    accuracy                           1.00       154
   macro avg       1.00      1.00      1.00       154
weighted avg       1.00      1.00      1.00       154


Confusion Matrix:
[[65  0]
 [ 0 89]]
Fold 2 - DNN ESA Override:

Classification Report:
              prec

C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

5/5 [==============================] - 0s 3ms/step
Fold 2 - DNN ESA Penalized:

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        65
           1       1.00      1.00      1.00        89

    accuracy                           1.00       154
   macro avg       1.00      1.00      1.00       154
weighted avg       1.00      1.00      1.00       154


Confusion Matrix:
[[65  0]
 [ 0 89]]
5/5 [==============================] - 0s 4ms/step
Fold 3 - DNN Baseline:

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        65
           1       1.00      0.99      0.99        89

    accuracy                           0.99       154
   macro avg       0.99      0.99      0.99       154
weighted avg       0.99      0.99      0.99       154


Confusion Matrix:
[[65  0]
 [ 1 88]]
Fold 3 - DNN ESA Override:

Classification Report:
              prec

...

C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

5/5 [==============================] - 0s 3ms/step
Fold 4 - DNN ESA Penalized:

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.98        65
           1       0.97      1.00      0.98        89

    accuracy                           0.98       154
   macro avg       0.98      0.98      0.98       154
weighted avg       0.98      0.98      0.98       154


Confusion Matrix:
[[62  3]
 [ 0 89]]
5/5 [==============================] - 0s 2ms/step
Fold 5 - DNN Baseline:

Classification Report:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        64
           1       1.00      0.96      0.98        89

    accuracy                           0.97       153
   macro avg       0.97      0.98      0.97       153
weighted avg       0.98      0.97      0.97       153


Confusion Matrix:
[[64  0]
 [ 4 85]]
Fold 5 - DNN ESA Override:

Classification Report:
              prec

C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

5/5 [==============================] - 0s 3ms/step
Fold 5 - DNN ESA Penalized:

Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98        64
           1       1.00      0.98      0.99        89

    accuracy                           0.99       153
   macro avg       0.98      0.99      0.99       153
weighted avg       0.99      0.99      0.99       153


Confusion Matrix:
[[64  0]
 [ 2 87]]

--- Final Evaluation (DNN Baseline) ---

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       324
           1       0.99      0.98      0.99       445

    accuracy                           0.99       769
   macro avg       0.99      0.99      0.99       769
weighted avg       0.99      0.99      0.99       769


Confusion Matrix:
[[321   3]
 [  7 438]]
{'accuracy': 0.9869960988296489, 'precision': 0.9931972789115646, 'recall': 0.9842696629213483, 'f

C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

# CVAR

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Loss
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
import time

In [7]:
def esa_score(phi, alpha):
    return np.dot(alpha, phi)

def threshold_crossing_rate(esa_baseline, esa_moral, tau):
    crossed = (esa_baseline < tau) & (esa_moral >= tau)
    return np.mean(crossed)

def moral_win_rate(esa_baseline, esa_moral):
    return np.mean(esa_moral > esa_baseline)

def esa_difference(esa_baseline, esa_moral):
    return np.mean(esa_moral - esa_baseline)

In [8]:
def evaluate_classification(y_true, y_pred):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1': f1_score(y_true, y_pred),
        'ROC_AUC': roc_auc_score(y_true, y_pred)
    }

In [14]:
class CVaRLoss(Loss):
    def __init__(self, alpha, phi_tensor, tau_tensor, lam=1):
        super().__init__()
        self.alpha = K.constant(alpha, dtype='float32')
        self.phi_tensor = K.constant(phi_tensor, dtype='float32')
        self.tau_tensor = K.constant(tau_tensor, dtype='float32')
        self.lam = lam

    def call(self, y_true, y_pred):
        y_true = K.cast(y_true, dtype='float32')
        bce = K.binary_crossentropy(y_true, y_pred)
        esa = K.sum(self.phi_tensor * self.alpha, axis=1)
        # Penalize low ESA values below CST threshold
        tail_penalty = K.relu(self.tau_tensor - esa)
        return bce + self.lam * tail_penalty

In [10]:
def build_dnn(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_dim=input_dim),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    return model

In [11]:
df = pd.read_csv('grad admission - ethics.csv')
    
y = df['accept_status_moral']
X = df.drop(columns=['accept_status', 'ESA', 'CST', 'severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int', 'accept_status_moral'])

esa_features = df[['severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int']].values
tau = df['CST'].values
phi = df[['severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int']].astype('float32').values
alpha = np.array([0.4, 0.2, 0.3, 0.0, 0.0, 0.1], dtype='float32')

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [12]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
dnn_cvar_preds = np.zeros(len(X))

In [15]:
#start timer
start_time = time.perf_counter()
for fold, (train_idx, val_idx) in enumerate(skf.split(X_scaled, y)):
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    phi_train = phi[train_idx]
    tau_train = tau[train_idx]

    model = build_dnn(X_train.shape[1])
    model.compile(optimizer=Adam(0.001), loss=CVaRLoss(alpha, phi_train, tau_train, lam=1.0))
    model.fit(X_train, y_train, epochs=30, batch_size=32, verbose=0, callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])
    y_val_pred = (model.predict(X_val).flatten() >= 1).astype(int)
    dnn_cvar_preds[val_idx] = y_val_pred
    print(f"Fold {fold+1} - DNN CVaR Loss:")
    print(evaluate_classification(y_val, y_val_pred))
    
end_time = time.perf_counter()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.6f} seconds")

metrics = evaluate_classification(y_val, y_val_pred)
print(metrics)

5/5 [==============================] - 0s 3ms/step
Fold 1 - DNN CVaR Loss:
{'Accuracy': 0.5974025974025974, 'Precision': 1.0, 'Recall': 0.30337078651685395, 'F1': 0.46551724137931033, 'ROC_AUC': 0.651685393258427}
5/5 [==============================] - 0s 4ms/step
Fold 2 - DNN CVaR Loss:
{'Accuracy': 0.5714285714285714, 'Precision': 1.0, 'Recall': 0.25842696629213485, 'F1': 0.41071428571428575, 'ROC_AUC': 0.6292134831460674}


5/5 [==============================] - 0s 3ms/step
Fold 3 - DNN CVaR Loss:
{'Accuracy': 0.5714285714285714, 'Precision': 1.0, 'Recall': 0.25842696629213485, 'F1': 0.41071428571428575, 'ROC_AUC': 0.6292134831460674}
5/5 [==============================] - 0s 3ms/step
Fold 4 - DNN CVaR Loss:
{'Accuracy': 0.5064935064935064, 'Precision': 1.0, 'Recall': 0.14606741573033707, 'F1': 0.25490196078431376, 'ROC_AUC': 0.5730337078651685}


5/5 [==============================] - 0s 2ms/step
Fold 5 - DNN CVaR Loss:
{'Accuracy': 0.5816993464052288, 'Precision': 1.0, 'Recall': 0.2808988764044944, 'F1': 0.4385964912280702, 'ROC_AUC': 0.6404494382022472}
Time taken: 19.327241 seconds
{'Accuracy': 0.5816993464052288, 'Precision': 1.0, 'Recall': 0.2808988764044944, 'F1': 0.4385964912280702, 'ROC_AUC': 0.6404494382022472}


# Adversarial Formulation

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Loss
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
import time

In [29]:
def esa_score(phi, alpha):
    return np.dot(phi, alpha)

def evaluate_classification(y_true, y_pred):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1': f1_score(y_true, y_pred),
        'ROC_AUC': roc_auc_score(y_true, y_pred)
    }

In [24]:
def adversary_loss(esa_true, esa_pred):
    return K.mean(K.square(esa_true - esa_pred))

In [25]:
def build_esa_adversary(input_dim):
    inputs = Input(shape=(input_dim,))
    x = Dense(32, activation='relu')(inputs)
    x = Dropout(0.3)(x)
    x = Dense(16, activation='relu')(x)
    esa_output = Dense(1, activation='linear', name='esa_output')(x)
    return Model(inputs=inputs, outputs=esa_output)

In [26]:
def build_adversarial_model(input_dim):
    # Main classifier
    input_layer = Input(shape=(input_dim,))
    x = Dense(64, activation='relu')(input_layer)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    output_class = Dense(1, activation='sigmoid', name='class_output')(x)

    # ESA adversary
    x_esa = Dense(16, activation='relu')(x)
    output_esa = Dense(1, activation='linear', name='esa_output')(x_esa)

    model = Model(inputs=input_layer, outputs=[output_class, output_esa])
    return model

In [30]:
df = pd.read_csv('credit_risk_dataset - ethics.csv')

# Initialize LabelEncoder
label_encoder = LabelEncoder() 

# Apply label encoding
df['person_home_ownership'] = label_encoder.fit_transform(df['person_home_ownership'])
df['loan_intent'] = label_encoder.fit_transform(df['loan_intent'])
df['loan_grade'] = label_encoder.fit_transform(df['loan_grade'])
df['cb_person_default_on_file'] = label_encoder.fit_transform(df['cb_person_default_on_file'])

y = df['loan_status']
X = df.drop(columns=['loan_status', 'CST', 'severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int'])

phi = df[['severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int']].astype('float32').values
alpha = np.array([0.4, 0.2, 0.3, 0.0, 0.0, 0.1], dtype='float32')
ESA_target = esa_score(phi, alpha).reshape(-1, 1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [31]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
dnn_adv_preds = np.zeros(len(X))


In [32]:
#start timer
start_time = time.perf_counter()

for fold, (train_idx, val_idx) in enumerate(skf.split(X_scaled, y)):
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    ESA_train, ESA_val = ESA_target[train_idx], ESA_target[val_idx]

    model = build_adversarial_model(X_train.shape[1])
    model.compile(optimizer=Adam(0.001),
                  loss={'class_output': 'binary_crossentropy', 'esa_output': 'mse'},
                  loss_weights={'class_output': 1.0, 'esa_output': 0.5})

    model.fit(X_train, {'class_output': y_train, 'esa_output': ESA_train},
              epochs=30, batch_size=32, verbose=0,
              validation_split=0.1,
              callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])

    y_val_pred = (model.predict(X_val)[0].flatten() >= 0.5).astype(int)
    dnn_adv_preds[val_idx] = y_val_pred
    print(f"Fold {fold+1} - ESA Adversarial:")
    print(evaluate_classification(y_val, y_val_pred))
    
end_time = time.perf_counter()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.6f} seconds")

metrics = evaluate_classification(y_val, y_val_pred)
print(metrics)

204/204 [==============================] - 0s 786us/step
Fold 1 - ESA Adversarial:
{'Accuracy': 0.9053245358293693, 'Precision': 0.9197080291970803, 'Recall': 0.620253164556962, 'F1': 0.7408651826963462, 'ROC_AUC': 0.8025701544080198}
204/204 [==============================] - 0s 850us/step
Fold 2 - ESA Adversarial:
{'Accuracy': 0.9165131982811541, 'Precision': 0.9507186858316222, 'Recall': 0.6511954992967651, 'F1': 0.7729549248747913, 'ROC_AUC': 0.8208863244422577}
204/204 [==============================] - 0s 896us/step
Fold 3 - ESA Adversarial:
{'Accuracy': 0.907305095150399, 'Precision': 0.9225206611570248, 'Recall': 0.6279887482419128, 'F1': 0.7472803347280335, 'ROC_AUC': 0.8066327722363863}
204/204 [==============================] - 0s 1ms/step
Fold 4 - ESA Adversarial:
{'Accuracy': 0.9102209944751382, 'Precision': 0.9106090373280943, 'Recall': 0.6523574947220268, 'F1': 0.7601476014760147, 'ROC_AUC': 0.8172484235141046}
204/204 [==============================] - 0s 1ms/step
Fold 